# Taller de ✨🦆redes neuronales🦆✨

### Vamos a entrenar una red neuronal simple utilizando Keras (un toolkit basado en TensorFlow) y la usaremos para reconocer los números manuscritos del corpus MNIST 

Pero antes que nada, importamos todo lo necesario para poder trabajar:

In [ ]:
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from keras.layers import Reshape
from keras.callbacks import LearningRateScheduler as LRS

Aquí definimos los "hiperparámetros" de la red:
- Tamaño de batch, que són el número de muestras que se procesan cada vez antes de actualizar los pesos de la red
- Épocas, que representan el número máximo de veces que veremos el conjunto de entrenamiento por completo
- Número de clases entre las que nuestro modelo ha de clasificar. En el caso de MNIST, son 10 (números del 0 al 9)

In [ ]:
batch_size = 512
epochs = 100
num_classes = 10

A continuación, prepararemos los datos 💾
Para ello, descargamos MNIST i tratamos un poquitín los datos:
- Reconstruimos las imágenes (que originalmente vienen en vectores de 784 dimensiones) en matrizes de 28x28 (para verlos como imágenes). Esto se hace primero que nada por conveniencia, por si se quieren visualizar los datos, además de por si se quiere hacer algún tipo de data augmentation mediante paneos, rotaciones...
- Codificamos los números a floats de 32 bits para que sean más manejables por la red y los dividimos entre 255 para escalarlos a valores entre 0 y 1.
- Convertimos las etiquetas (numéricas) a una representación interna de Keras para poder trabajar más cómodamente

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print("Training set", x_train.shape)
print("test set", x_test.shape)

x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

def show_img(img):
  import matplotlib.pyplot as plt

  img = np.array(img)
  plt.imshow(img, cmap= 'gray')

show_img(x_test[0])

x_train /= 255
x_test /= 255

y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test , num_classes)

Ahora ya por fin vamos a definir el modelo 🤖 🎉 📈

Utilizaremos la API secuencial de Keras, con la que podemos añadir capas simplemente poniendo model.add(*capa*)

Las capas que hemos añadido son de dos tipos
- Reshape, para reconstruir de nuevo las imágenes en vectores (la entrada de una red normal)
- Dense, que implementa una capa densa de neuronas con un número de unidades igual a su primer parámetro. La activación de la capa se especifica mediante texto con el parámetro _activation_

Podéis ver que, en este caso, he construido una red con tres capas de 1024, 1024 y 10 neuronas respectivamente, aunque no es necesario seguir esta tendencia. Se pueden generar redes en forma de "embudo", simulando una arquitectura "encoder-decoder"... El límite es vuestra imaginación xdd

Finalmente, mostramos un "resumen" del modelo, donde vemos todas las capas y parámetros que hemos creado.

In [ ]:
model = Sequential()

model.add(Reshape(target_shape=(784,), input_shape=(28,28,1)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

En esta celda definimos el optimizador que utilizaremos para entrenar la red (en este caso, Adam, que funciona muy bien) y "compilamos" el modelo especificando la función de pérdida, el optimizador a utilizar y las métricas que queremos monitorizar.

Dado que estamos manejando un problema de clasificación, utilizamos la entropía cruzada como función de pérdida. Para tareas de regresión (generación de imágenes, por ejemplo), en cambio, se puede utilizar el error cuadrático medio.

In [ ]:
opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

Aquí definimos un "scheduler" para el learning rate. Este paso no es obligatorio, puesto que podemos utilizar un ratio de aprendizaje fijo durante todo el entrenamiento, pero es una buena idea ir reduciendo este parámetro durante el entrenamiento para ser capaces de alcanzar mínimos más "estrechos".

También podemos utilizar esta celda para añadir otras "callbacks", que simplemente son funciones que se ejecutan al final de cada época para controlar factores como el ratio de aprendizaje, paradas tempranas para evitar largas ejecuciones sin mejoras o guardar "checkpoints" de los mejores modelos hasta el momento.

In [ ]:
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * np.exp(-0.1)

lrs = LRS(scheduler)
callbacks = [lrs]

Finalmente, entrenamos el modelo. Esto simplemente lo hacemos ejecutando la función "fit". En este paso, pasamos todos los parámetros necesarios que hemos estado preparando (datos, tamaño de batch, épocas, callbacks...) y evaluamos el último modelo respecto al conjunto de test.

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks)

score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Finalmente, imprimimos la evolución de la tasa de aciertos a lo largo del proceso de entrenamiento

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Y esto es todo!! :)

Ahora, como ejercicio, tenéis que tratar de aumentar esa tasa de aciertos tanto como podáis (tranquilos, no es un concurso). Para ello, podéis modificar los hiperparámetros que queráis, aumentar el número de capas del modelo o su número de unidades, añadir técnicas como dropout, ruido gaussiano, normalización de pesos por batch... Además de todas las cosas que se os ocurran :D

Si tenéis dudas, me preguntáis. Mi mejor intento ha obtenido una tasa máxima de 0.9941 de acierto en test. Ánimo :)